In [14]:
#!pip install tensorflow-gpu
#Without this, GPU memory no longer caps at 0.1 GB (not utilized; CPU used). The default tensorflow gpu installation on colab machines is correct.

In [15]:
!pip install git+https://github.com/Joeclinton1/google-images-download.git

  Cloning https://github.com/Joeclinton1/google-images-download.git to /tmp/pip-req-build-ld368s_x
  Running command git clone -q https://github.com/Joeclinton1/google-images-download.git /tmp/pip-req-build-ld368s_x
  Created wheel for google-images-download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=15833 sha256=a37341afc87ffbce2a219dfbdabdddb9a16da7030b9a79dc9c51af9a82281bef
  Stored in directory: /tmp/pip-ephem-wheel-cache-polvz6qb/wheels/1a/36/c9/53bd827bc241b69fe47b90731625459627f82694ecd8a88273
Successfully built google-images-download


In [16]:
from zipfile import ZipFile

In [17]:
with ZipFile("resizedImages.zip",'r') as zip1:
  zip1.extractall()

In [18]:
"""
Created on Wed May  5 21:12:02 2021

@author: Rajat Bakshi
"""
import h5py
import numpy as np
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
import json
import requests
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.layers import UpSampling2D, Conv2D
from keras.optimizers import Adam
from PIL import Image, ImageOps
from pathlib import Path
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt

from google_images_download import google_images_download

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


print(tf.config.list_physical_devices('GPU'))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 652317148362666000
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8431595303887800435
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 1327649695300519500
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7250706432
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1247899391922059433
physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"
]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


The following block of code downloads images using google images

In [19]:
'''

response=google_images_download.googleimagesdownload()
arguments={"keywords":"snowy mountains", "limit": 100, "print_urls": False}
response.download(arguments)

arguments={"keywords":"desert mountains", "limit": 100, "print_urls": False}
response.download(arguments)

arguments={"keywords":"green mountains", "limit": 100, "print_urls": False}
response.download(arguments)

arguments={"keywords":"high rocky mountains", "limit": 100, "print_urls": False}
response.download(arguments)

arguments={"keywords":"high mountains", "limit": 100, "print_urls": False}
response.download(arguments)

arguments={"keywords":"scottish highland mountains", "limit": 100, "print_urls": False}
response.download(arguments)



def get_all_images(folder, ext):

    all_files = []
    #Iterate through all files in folder
    for file in os.listdir(folder):
        #Get the file extension
        _,  file_ext = os.path.splitext(file)
        
        #If file is of given extension, get it's full path and append to list
        if ext in file_ext:
            full_file_path = os.path.join(folder, file)
            all_files.append(full_file_path)

    #Get list of all files
    return all_files

imagePaths1 = get_all_images('D:\MountainsGeneratorGAN\downloads\desertMountains', 'jpg')
imagePaths2 = get_all_images('D:\MountainsGeneratorGAN\downloads\greenMountains', 'jpg')
imagePaths3 = get_all_images('D:\MountainsGeneratorGAN\downloads\highMountains', 'jpg')
imagePaths4 = get_all_images('D:\MountainsGeneratorGAN\downloads\highRockyMountains', 'jpg')
imagePaths5 = get_all_images('D:\MountainsGeneratorGAN\downloads\scottishHighlandMountains', 'jpg')
imagePaths6 = get_all_images('D:\MountainsGeneratorGAN\downloads\snowyMountains', 'jpg')

imagePaths=[]
imagePaths.extend(imagePaths1)
imagePaths.extend(imagePaths2)
imagePaths.extend(imagePaths3)
imagePaths.extend(imagePaths4)
imagePaths.extend(imagePaths5)
imagePaths.extend(imagePaths6)

if not os.path.exists('resizedImages'):
    os.makedirs('resizedImages')
    
imageCount=0
for imagePath in imagePaths:
    imageCount+=1
    image = Image.open(imagePath)
    image.thumbnail((200, 200))
    image.save('resizedImages/mtnImage'+str(imageCount)+'.jpg')
    mirroredImage=ImageOps.mirror(image)
    mirroredImage.save('resizedImages/mtnImage'+str(imageCount)+'mirrored.jpg')

'''

'\n\nresponse=google_images_download.googleimagesdownload()\narguments={"keywords":"snowy mountains", "limit": 100, "print_urls": False}\nresponse.download(arguments)\n\narguments={"keywords":"desert mountains", "limit": 100, "print_urls": False}\nresponse.download(arguments)\n\narguments={"keywords":"green mountains", "limit": 100, "print_urls": False}\nresponse.download(arguments)\n\narguments={"keywords":"high rocky mountains", "limit": 100, "print_urls": False}\nresponse.download(arguments)\n\narguments={"keywords":"high mountains", "limit": 100, "print_urls": False}\nresponse.download(arguments)\n\narguments={"keywords":"scottish highland mountains", "limit": 100, "print_urls": False}\nresponse.download(arguments)\n\n\n\ndef get_all_images(folder, ext):\n\n    all_files = []\n    #Iterate through all files in folder\n    for file in os.listdir(folder):\n        #Get the file extension\n        _,  file_ext = os.path.splitext(file)\n        \n        #If file is of given extension,

In [26]:
keras.backend.clear_session() #clearing VRAM

The below block of code contains training parameters 

In [27]:
DATA_PATH='resizedImages'

GENERATE_RES = 5 # Generation resolution factor 
# (1=32, 2=64, 3=96, 4=128, etc.)
GENERATE_SQUARE = 32 * GENERATE_RES # rows/cols (should be square)
IMAGE_CHANNELS = 3

PREVIEW_ROWS = 7
PREVIEW_COLS = 7
PREVIEW_MARGIN = 16

# Size vector to generate images from
SEED_SIZE = 1000
EPOCHS = 4000
BATCH_SIZE = 32
BUFFER_SIZE = 958

In [28]:
training_binary_path = os.path.join(DATA_PATH,
        f'training_data_{GENERATE_SQUARE}_{GENERATE_SQUARE}.npy')

print(f"Looking for file: {training_binary_path}")

if not os.path.isfile(training_binary_path):
    print("Loading training images...")
    
    training_data = []
    for filename in tqdm(os.listdir(DATA_PATH)):
        path = os.path.join(DATA_PATH,filename)
        if os.path.isfile(path):
          image = Image.open(path).resize((GENERATE_SQUARE,
                GENERATE_SQUARE),Image.ANTIALIAS)
          training_data.append(np.asarray(image))
    
    training_data = np.reshape(training_data,(-1,GENERATE_SQUARE,
              GENERATE_SQUARE,IMAGE_CHANNELS))
    training_data = training_data.astype(np.float32)
    training_data = training_data / 127.5 - 1.

    print("Saving training image binary...")
    np.save(training_binary_path,training_data)


else:
  print("Loading previous training pickle...")
  training_data = np.load(training_binary_path)


train_dataset = tf.data.Dataset.from_tensor_slices(training_data) \
    .shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

  7%|▋         | 69/960 [00:00<00:01, 684.79it/s]

Looking for file: resizedImages/training_data_160_160.npy
Loading training images...


100%|██████████| 960/960 [00:01<00:00, 712.17it/s]


Saving training image binary...


Generator and Discriminator models:

In [29]:
def build_generator(seed_size, channels):
    
    model=Sequential()
    
    model.add(Dense(4*4*256,activation="relu",input_dim=seed_size))
    model.add(Reshape((4,4,256)))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
   
    # Output resolution, additional upsampling
    model.add(UpSampling2D())
    model.add(Conv2D(128,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    if GENERATE_RES>1:
      model.add(UpSampling2D(size=(GENERATE_RES,GENERATE_RES)))
      model.add(Conv2D(128,kernel_size=3,padding="same"))
      model.add(BatchNormalization(momentum=0.8))
      model.add(Activation("relu"))

    # Final CNN layer
    model.add(Conv2D(channels,kernel_size=3,padding="same"))
    model.add(Activation("tanh"))

    return model

def build_discriminator(image_shape):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, 
                     padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

In [ ]:
def save_images(cnt,noise):
  image_array = np.full(( 
      PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 
      PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 3), 
      255, dtype=np.uint8)
  
  generated_images = generator.predict(noise)

  generated_images = 0.5 * generated_images + 0.5

  image_count = 0
  for row in range(PREVIEW_ROWS):
      for col in range(PREVIEW_COLS):
        r = row * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        c = col * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE] \
            = generated_images[image_count] * 255
        image_count += 1

          
  output_path = os.path.join(DATA_PATH,'output')
  if not os.path.exists(output_path):
      os.makedirs(output_path)
  
  filename = os.path.join(output_path,f"train-{cnt}.png")
  im = Image.fromarray(image_array)
  im.save(filename)


generator = build_generator(SEED_SIZE, IMAGE_CHANNELS)

noise = tf.random.normal([1, SEED_SIZE])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0])

image_shape = (GENERATE_SQUARE,GENERATE_SQUARE,IMAGE_CHANNELS)

discriminator = build_discriminator(image_shape)
decision = discriminator(generated_image)
print (decision)


# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(7e-5,0) #learning rate, momentum
discriminator_optimizer = tf.keras.optimizers.Adam(10e-5,0)



# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
  seed = tf.random.normal([BATCH_SIZE, SEED_SIZE])

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

    generated_images = generator(seed, training=True)
    
            
    real_output = discriminator(images, training=True)
    fake_output = discriminator(generated_images, training=True)
        
    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)
    

    gradients_of_generator = gen_tape.gradient(\
        gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(\
        disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(
        gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(
        gradients_of_discriminator, 
        discriminator.trainable_variables))
  return gen_loss,disc_loss

def train(dataset, epochs):
  fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, 
                                       SEED_SIZE))
  
  for epoch in range(epochs):
    epoch_start = time.time()

    gen_loss_list = []
    disc_loss_list = []

    for image_batch in dataset:
      t = train_step(image_batch)
      gen_loss_list.append(t[0])
      disc_loss_list.append(t[1])

    g_loss = sum(gen_loss_list) / len(gen_loss_list)
    d_loss = sum(disc_loss_list) / len(disc_loss_list)

    epoch_elapsed = time.time()-epoch_start
    print (f'Epoch {epoch+1}, gen loss={g_loss},disc loss={d_loss},'\
           ' {hms_string(epoch_elapsed)}')
    save_images(epoch,fixed_seed)
    
    
train(train_dataset, EPOCHS)

generator.save(os.path.join(DATA_PATH,"landscape_generator.h5"))

tf.Tensor([[0.5017804]], shape=(1, 1), dtype=float32)
Epoch 1, gen loss=2.949155569076538,disc loss=2.419538974761963, {hms_string(epoch_elapsed)}
Epoch 2, gen loss=3.6130549907684326,disc loss=1.864182949066162, {hms_string(epoch_elapsed)}
Epoch 3, gen loss=3.438223123550415,disc loss=2.545619249343872, {hms_string(epoch_elapsed)}
Epoch 4, gen loss=3.643547773361206,disc loss=2.642657995223999, {hms_string(epoch_elapsed)}
Epoch 5, gen loss=3.4602606296539307,disc loss=2.8057703971862793, {hms_string(epoch_elapsed)}
Epoch 6, gen loss=3.1209981441497803,disc loss=3.1440160274505615, {hms_string(epoch_elapsed)}
Epoch 7, gen loss=3.1248440742492676,disc loss=2.624812364578247, {hms_string(epoch_elapsed)}
Epoch 8, gen loss=3.067058563232422,disc loss=2.763556480407715, {hms_string(epoch_elapsed)}
Epoch 9, gen loss=3.0184173583984375,disc loss=2.4004366397857666, {hms_string(epoch_elapsed)}
Epoch 10, gen loss=3.145270824432373,disc loss=2.2853779792785645, {hms_string(epoch_elapsed)}
Epoch 